# Segmentation

In [2]:
import os
import json
from pathlib import Path

In [3]:
from src.dataset import SeaTurtleDataset, download_dataset
# from src.arcface import ArcFace
from src.utils import get_device

In [4]:
paths = download_dataset()
paths

Dataset downloaded and extracted to: /Users/nhut/.cache/kagglehub/datasets/wildlifedatasets/seaturtleid2022/versions/4


{'path': '/Users/nhut/.cache/kagglehub/datasets/wildlifedatasets/seaturtleid2022/versions/4',
 'images_path': '/Users/nhut/.cache/kagglehub/datasets/wildlifedatasets/seaturtleid2022/versions/4/turtles-data/data',
 'annotations_path': '/Users/nhut/.cache/kagglehub/datasets/wildlifedatasets/seaturtleid2022/versions/4/turtles-data/data/annotations.json',
 'metadata': '/Users/nhut/.cache/kagglehub/datasets/wildlifedatasets/seaturtleid2022/versions/4/turtles-data/data/metadata.csv',
 'metadata_splits': '/Users/nhut/.cache/kagglehub/datasets/wildlifedatasets/seaturtleid2022/versions/4/turtles-data/data/metadata_splits.csv'}

In [ ]:
# original annotations file
# annotations_path = paths["annotations_path"]

# sub-dataset annotations file
annotations_path = "./data/seaturtleid2022-subset/annotations.json"

with open(annotations_path, "r") as f:
    coco = json.load(f)

print("Images:", len(coco["images"]))
print("Annotations:", len(coco["annotations"]))
print("Categories:", coco["categories"])

display(coco.keys())
display(coco['images'][0])
display(coco['annotations'][0].keys())

Images: 620
Annotations: 2591
Categories: [{'id': 1, 'name': 'turtle', 'supercategory': ''}, {'id': 2, 'name': 'flipper', 'supercategory': ''}, {'id': 3, 'name': 'head', 'supercategory': ''}]


dict_keys(['licenses', 'info', 'categories', 'images', 'annotations'])

{'id': 448,
 'width': 2000,
 'height': 1500,
 'file_name': 'images/t023/AKOFtQyjtX.JPG',
 'license': 0,
 'flickr_url': '',
 'coco_url': '',
 'date_captured': 0,
 'timestamp': '2011:07:30 13:49:47',
 'identity': 't023'}

dict_keys(['id', 'image_id', 'category_id', 'segmentation', 'area', 'bbox', 'iscrowd', 'attributes'])